In [ ]:
import numpy as np
import cv2


def draw_line(event, x, y, flags, param):
    global prv_x, prv_y, prv_KFx, prv_KFy, Pos_x, Pos_y, KF_x
    if event == cv2.EVENT_MOUSEMOVE:
        # 현재 마우스 위치 저장
        Pos_x, Pos_y = x, y

        # 실제 위치(빨간 선)와 예측 위치(파란 선) 그리기
        if prv_x != 0 and prv_y != 0:
            cv2.line(img, (prv_x, prv_y), (x, y), (255, 0, 0), 3)
            cv2.line(
                img,
                (int(prv_KFx), int(prv_KFy)),
                (int(KF_x[0, 0]), int(KF_x[2, 0])),
                (0, 0, 255),
                3,
            )

        prv_x, prv_y = x, y
        prv_KFx, prv_KFy = KF_x[0, 0], KF_x[2, 0]


def get_sensor_data():
    global v, Pos_x, Pos_y

    # 센서 노이즈 추가
    v_x = np.random.normal(0, 5)
    v_y = np.random.normal(0, 5)

    # 센서 데이터 생성
    measurement_x = Pos_x + v_x
    measurement_y = Pos_y + v_y
    return np.array([[measurement_x], [measurement_y]], np.float64)


def measurement_update(est_x, est_P, measurement_z):
    global H, R
    # 칼만 이득 계산
    temp = (H @ est_P @ H.T) + R
    K = est_P @ H.T @ np.linalg.inv(temp)

    # 상태 업데이트
    x = est_x + K @ (measurement_z - H @ est_x)
    P = est_P - K @ H @ est_P
    return x, P


def state_prediction(prev_x, prev_P):
    global A, Q
    # 상태와 공분산 예측
    est_x = A @ prev_x
    est_P = A @ prev_P @ A.T + Q
    return est_x, est_P


# 초기값 설정
prv_x, prv_y, prv_KFx, prv_KFy = 0, 0, 0, 0
window_width = 1280
window_height = 760
dt = 0.03

# Kalman Filter 시스템 모델
A = np.array([[1, dt, 0, 0], [0, 1, 0, 0], [0, 0, 1, dt], [0, 0, 0, 1]], np.float64)
H = np.array([[1, 0, 0, 0], [0, 0, 1, 0]], np.float64)
Q = 1.0 * np.eye(4)  # 프로세스 노이즈 공분산
R = np.array([[50, 0], [0, 50]], np.float64)  # 측정 노이즈 공분산
KF_x = np.array([[0], [0], [0], [0]], np.float64)  # 초기 상태
P = 100 * np.eye(4)  # 초기 공분산
Pos_x, Pos_y = 0, 0

# 캔버스 생성
img = np.zeros((window_height, window_width, 3), np.uint8)
cv2.namedWindow("Cursor Tracking", cv2.WINDOW_AUTOSIZE)
cv2.setMouseCallback("Cursor Tracking", draw_line)

# 메인 루프
while True:
    cv2.imshow("Cursor Tracking", img)

    # 센서 데이터 가져오기
    z = get_sensor_data()

    # 상태 예측 및 측정 업데이트
    est_x, est_P = state_prediction(KF_x, P)
    KF_x, P = measurement_update(est_x, est_P, z)

    # ESC 키 입력으로 종료
    if cv2.waitKey(int(dt * 1000)) & 0xFF == 27:
        break

cv2.destroyAllWindows()

In [5]:
import numpy as np
import cv2

# 마우스 이동을 감지하고 실제 위치(파란 선)와 칼만 필터의 예측 위치(빨간 선)를 화면에 그리는 함수
def draw_line(event, x, y, flags, param):
    global prv_x, prv_y, prv_KFx, prv_KFy, Pos_x, Pos_y, KF_x  # 전역 변수 사용
    if event == cv2.EVENT_MOUSEMOVE:  # 마우스가 움직일 때 이벤트 발생
        # 현재 마우스 위치를 기록
        Pos_x, Pos_y = x, y

        # 이전 마우스 위치와 현재 위치 사이를 선으로 연결하여 실제 경로를 시각화 (파란색 선)
        if prv_x != 0 and prv_y != 0:
            cv2.line(img, (prv_x, prv_y), (x, y), (255, 0, 0), 3)  # (파란 선: 실제 마우스 경로 (BGR임)
            cv2.line(
                img,
                (int(prv_KFx), int(prv_KFy)),
                (int(KF_x[0, 0]), int(KF_x[2, 0])),
                (0, 0, 255),
                3,
            )  # 파란 선: 칼만 필터의 예측 경로

        # 현재 위치를 이전 위치로 업데이트
        prv_x, prv_y = x, y
        prv_KFx, prv_KFy = KF_x[0, 0], KF_x[2, 0]


# 센서 데이터를 생성하는 함수 (노이즈를 포함한 측정값 반환)
def get_sensor_data():
    global v, Pos_x, Pos_y

    # 측정 노이즈를 가우시안 분포로 생성 (x와 y 축 각각)
    v_x = np.random.normal(0, 5)  # x 축 노이즈
    v_y = np.random.normal(0, 5)  # y 축 노이즈

    # 노이즈를 더해 센서 데이터를 생성 (현재 마우스 위치 + 노이즈)
    measurement_x = Pos_x + v_x
    measurement_y = Pos_y + v_y
    return np.array([[measurement_x], [measurement_y]], np.float64)


# 칼만 필터의 측정 업데이트 단계
def measurement_update(est_x, est_P, measurement_z):
    global H, R
    # 칼만 이득(K) 계산
    temp = (H @ est_P @ H.T) + R
    K = est_P @ H.T @ np.linalg.inv(temp)  # 이득 계산

    # 상태 업데이트: 측정값(z)과 예측값의 차이를 이용해 보정
    x = est_x + K @ (measurement_z - H @ est_x)

    # 공분산 업데이트: 보정된 상태에서의 불확실성 감소
    P = est_P - K @ H @ est_P
    return x, P


# 칼만 필터의 상태 예측 단계
def state_prediction(prev_x, prev_P):
    global A, Q
    # 상태를 다음 단계로 예측
    est_x = A @ prev_x

    # 공분산을 다음 단계로 예측
    est_P = A @ prev_P @ A.T + Q
    return est_x, est_P


# 초기값 설정
prv_x, prv_y, prv_KFx, prv_KFy = 0, 0, 0, 0  # 이전 위치 초기화
window_width = 1280  # 창 너비
window_height = 760  # 창 높이
dt = 0.03  # 시간 간격 (초 단위)

# 칼만 필터 시스템 모델 정의
A = np.array([[1, dt, 0, 0], [0, 1, 0, 0], [0, 0, 1, dt], [0, 0, 0, 1]], np.float64)  # 상태 전이 행렬
H = np.array([[1, 0, 0, 0], [0, 0, 1, 0]], np.float64)  # 측정 행렬
Q = 1.0 * np.eye(4)  # 프로세스 노이즈 공분산 행렬 (시스템 모델의 불확실성)
R = np.array([[50, 0], [0, 50]], np.float64)  # 측정 노이즈 공분산 행렬 (센서 노이즈)
KF_x = np.array([[0], [0], [0], [0]], np.float64)  # 초기 상태 (x, x', y, y')
P = 100 * np.eye(4)  # 초기 상태 공분산
Pos_x, Pos_y = 0, 0  # 마우스의 초기 위치

# 캔버스 생성
img = np.zeros((window_height, window_width, 3), np.uint8)  # 빈 이미지 생성
cv2.namedWindow("Cursor Tracking", cv2.WINDOW_AUTOSIZE)  # 창 생성
cv2.setMouseCallback("Cursor Tracking", draw_line)  # 마우스 이벤트 연결

# 메인 루프
while True:
    cv2.imshow("Cursor Tracking", img)  # 캔버스 화면에 표시

    # 센서 데이터를 생성
    z = get_sensor_data()

    # 칼만 필터 상태 예측과 측정 업데이트 단계 수행
    est_x, est_P = state_prediction(KF_x, P)
    KF_x, P = measurement_update(est_x, est_P, z)

    # ESC 키 입력 시 프로그램 종료
    if cv2.waitKey(int(dt * 1000)) & 0xFF == 27:
        break

cv2.destroyAllWindows()  # 창 닫기


KeyboardInterrupt: 